In [1]:
!pip install sklearn progressbar2

     |████████████████████████████████| 6.8 MB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 298 kB 17.9 MB/s eta 0:00:01
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=2397 sha256=1d7aa36e87b1754da535db927b35cdce8d0525cb69fddef54686ea63c1563a95
  Stored in directory: /root/.cache/pip/wheels/23/9d/42/5ec745cbbb17517000a53cecc49d6a865450d1f5cb16dc8a9c
Successfully built sklearn
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import os
import h5py
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import tensorflow as tf
from tensorflow import keras
import gc
from sys import getsizeof
import datetime
import random
import math
from sklearn.metrics import classification_report, confusion_matrix
import progressbar

In [3]:
GB = 1
gpus = tf.config.experimental.list_physical_devices('GPU')
#The variable GB is the memory size you want to use.
config = [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=(1024*GB))]
if gpus:
  # Restrict TensorFlow to only allocate 1*X GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(gpus[0], config)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)
np.set_printoptions(threshold=40*40)


In [4]:
def load_data(res, source_labels, dataset='train', size=1., verbosity = False):
    data = np.zeros((0, res, res, res))
    labels = []
    for i, label in enumerate(source_labels):
        filename = './output/' + label + '/output-' + dataset + '-' + str(res) + '.h5'
        file = h5py.File(filename, 'r')
        tensor_len = len(file['tensor'])
        picked_ids = random.sample(range(tensor_len), math.ceil(tensor_len * size))
        if verbosity:
            print(filename)
            print('Picking ' + str(math.ceil(tensor_len * size)) + ' from ' + str(len(file['tensor'])) )
        data = np.concatenate((data, [file['tensor'][index] for index in picked_ids]))
        labels = np.concatenate((labels, np.full(len(picked_ids), i)))
        file.close()
        gc.collect()
    rng_state = np.random.get_state()
    np.random.shuffle(data)
    np.random.set_state(rng_state)
    np.random.shuffle(labels)
    return (data, labels)


In [54]:
def prepare_model(res, num_classes, verbosity = 0):
    model = keras.Sequential([
        keras.layers.Flatten(input_shape=(res, res, res)),
        keras.layers.Dense(res*2, activation='relu'),
        keras.layers.Dense(res/2, activation='sigmoid'),
        keras.layers.Dropout(.2, input_shape=(2,)),
        keras.layers.Dense(num_classes, activation='softmax')
    ])

    learning_rate = 0.0001

    model.compile(optimizer=keras.optimizers.Adam(lr=learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    if (verbosity == 1):
        print(model.summary())
    return model
def prepare_data(data, num_classes):
    return data
model_type = 'dense'
prepare_model(24, 20, 1)

Model: "sequential_1671"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1672 (Flatten)       (None, 13824)             0         
_________________________________________________________________
dense_5016 (Dense)           (None, 48)                663600    
_________________________________________________________________
dense_5017 (Dense)           (None, 12)                588       
_________________________________________________________________
dropout_1672 (Dropout)       (None, 12)                0         
_________________________________________________________________
dense_5018 (Dense)           (None, 20)                260       
Total params: 664,448
Trainable params: 664,448
Non-trainable params: 0
_________________________________________________________________
None


In [5]:
def prepare_model(res, num_classes, verbosity = 0):
    model = keras.Sequential([
        keras.layers.Conv3D(
                res, 
                kernel_size=(3), 
                activation='relu', 
                kernel_initializer='he_uniform', 
                data_format="channels_last",
                input_shape=(res, res, res, num_classes)
            ),        
        keras.layers.MaxPooling3D(pool_size=(2, 2, 2)),
        keras.layers.Flatten(),
        keras.layers.Dense(res/2, activation='relu'),
        keras.layers.Dense(num_classes, activation='softmax')
    ])

    learning_rate = 0.0001

    model.compile(optimizer=keras.optimizers.Adam(lr=learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    if (verbosity == 1):
        print(model.summary())
    return model
def prepare_data(data, num_classes):
    return keras.utils.to_categorical(data, num_classes)
model_type = 'conv3d'
prepare_model(24, 10, 1)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 22, 22, 22, 24)    6504      
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 11, 11, 11, 24)    0         
_________________________________________________________________
flatten (Flatten)            (None, 31944)             0         
_________________________________________________________________
dense (Dense)                (None, 12)                383340    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                130       
Total params: 389,974
Trainable params: 389,974
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
def train(model, training, training_labels, res, num_classes, verbosity = 0):
    log_dir = "logs/fit/" + str(res) + '-' + str(num_classes) + '-' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    if verbosity == 1:
        print("Log file " + log_dir)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    validation_split = 0.1
    epochs = 10 * num_classes
    
#    model.fit(
#        keras.utils.to_categorical(training, num_clases), 
#        training_labels, 
#        epochs=epochs)
    
    model.fit(
        prepare_data(training, num_classes), 
        training_labels, 
        epochs=epochs,
        verbose=verbosity,
        callbacks=[tensorboard_callback],
        validation_split=validation_split
    )

In [13]:
#Confusion Matrix and Classification Report
def print_confusion_matrix(test, test_labels, base_labels):
    Y_pred = model.predict(prepare_data(test, len(base_labels)), len(test))
    y_pred = np.argmax(Y_pred, axis=1)
    print('Confusion Matrix')
    print(confusion_matrix(test_labels, y_pred))
    #print('Classification Report')
    #print(classification_report(test_labels, y_pred, target_names=base_labels))

In [8]:
#base_labels = ['bathtub', 'bed', 'chair', 'desk', 'dresser', 'monitor', 'night_stand', 'sofa', 'table', 'toilet']
all_labels = ['airplane', 'bathtub', 'bed', 'bench', 'bookshelf', 'bottle', 'bowl', 'car', 'chair', 'cone', 
                 'cup', 'curtain', 'desk', 'door', 'dresser', 'flower_pot', 'glass_box', 'guitar','keyboard', 'lamp',
                 'laptop', 'mantel', 'monitor', 'night_stand', 'person', 'piano', 'plant', 
                 'radio', 'range_hood', 'sink', 'sofa', 'stairs', 'stool', 'table', 'tent', 'toilet', 'tv_stand', 
                 'vase', 'wardrobe', 'xbox']


In [ ]:
subclases = [10, 20, 30, 40]
resolutions = [24]#, 32]#, 48]
num_classes = 2

for i in range(len(all_labels)):
    for j in range(i + 1, len(all_labels)):
        base_labels = [all_labels[i]] + [all_labels[j]]
        print(i, j, base_labels)
        for n, res in enumerate(resolutions):
            training, training_labels = load_data(res, base_labels, 'train', 1, False)
            model = prepare_model(res, num_classes, 0)
            model_name = 'models/' + model_type + '-' + all_labels[i] + '-' + all_labels[j] + '.h5'
            train(model, training, training_labels, res, num_classes, 1)
            model.save(model_name)
            test, test_labels = load_data(res, base_labels, 'test', 1.0, False)
            test_loss, test_acc = model.evaluate(
                prepare_data(test, num_classes),  
                test_labels, 
                verbose=2
            )
            print('\nTest accuracy for ' + all_labels[i] + '-' + all_labels[j] + ' classes width res ' + str(res) + ':', test_acc)
            #print_confusion_matrix(test, test_labels, all_labels)
            gc.collect()          


0 1 ['airplane', 'bathtub']
Log file logs/fit/24-2-20200624-050044
Epoch 1/20
21/21 [==============================] - 19s 896ms/step - loss: 0.2094 - accuracy: 0.9286 - val_loss: 0.0807 - val_accuracy: 0.9595
Epoch 2/20
21/21 [==============================] - 19s 891ms/step - loss: 0.0288 - accuracy: 0.9954 - val_loss: 0.0113 - val_accuracy: 1.0000
Epoch 3/20
21/21 [==============================] - 19s 915ms/step - loss: 0.0105 - accuracy: 1.0000 - val_loss: 0.0052 - val_accuracy: 1.0000
Epoch 4/20
21/21 [==============================] - 19s 894ms/step - loss: 0.0053 - accuracy: 1.0000 - val_loss: 0.0037 - val_accuracy: 1.0000
Epoch 5/20
21/21 [==============================] - 19s 919ms/step - loss: 0.0040 - accuracy: 1.0000 - val_loss: 0.0037 - val_accuracy: 1.0000
Epoch 6/20
21/21 [==============================] - 19s 893ms/step - loss: 0.0029 - accuracy: 1.0000 - val_loss: 0.0021 - val_accuracy: 1.0000
Epoch 7/20
21/21 [==============================] - 19s 915ms/step - loss: 

Epoch 15/20
23/23 [==============================] - 21s 895ms/step - loss: 0.0617 - accuracy: 0.9736 - val_loss: 0.1051 - val_accuracy: 0.9625
Epoch 16/20
23/23 [==============================] - 21s 894ms/step - loss: 0.0580 - accuracy: 0.9805 - val_loss: 0.1151 - val_accuracy: 0.9500
Epoch 17/20
23/23 [==============================] - 21s 899ms/step - loss: 0.0645 - accuracy: 0.9666 - val_loss: 0.1062 - val_accuracy: 0.9625
Epoch 18/20
23/23 [==============================] - 21s 896ms/step - loss: 0.0606 - accuracy: 0.9750 - val_loss: 0.1221 - val_accuracy: 0.9625
Epoch 19/20
23/23 [==============================] - 21s 896ms/step - loss: 0.0626 - accuracy: 0.9777 - val_loss: 0.1113 - val_accuracy: 0.9625
Epoch 20/20
23/23 [==============================] - 21s 898ms/step - loss: 0.0539 - accuracy: 0.9750 - val_loss: 0.1059 - val_accuracy: 0.9500
4/4 - 0s - loss: 0.0503 - accuracy: 0.9833

Test accuracy for airplane-bench classes width res 24: 0.9833333492279053
0 4 ['airplane', '

Epoch 9/20
20/20 [==============================] - 20s 1s/step - loss: 0.1057 - accuracy: 0.9823 - val_loss: 0.0218 - val_accuracy: 1.0000
Epoch 10/20
20/20 [==============================] - 21s 1s/step - loss: 0.1132 - accuracy: 0.9823 - val_loss: 0.0224 - val_accuracy: 1.0000
Epoch 11/20
20/20 [==============================] - 21s 1s/step - loss: 0.1034 - accuracy: 0.9807 - val_loss: 0.0372 - val_accuracy: 1.0000
Epoch 12/20
20/20 [==============================] - 21s 1s/step - loss: 0.1044 - accuracy: 0.9758 - val_loss: 0.0788 - val_accuracy: 1.0000
Epoch 13/20
20/20 [==============================] - 21s 1s/step - loss: 0.1165 - accuracy: 0.9742 - val_loss: 0.0503 - val_accuracy: 1.0000
Epoch 14/20
20/20 [==============================] - 19s 935ms/step - loss: 0.1047 - accuracy: 0.9823 - val_loss: 0.0216 - val_accuracy: 1.0000
Epoch 15/20
20/20 [==============================] - 18s 884ms/step - loss: 0.0971 - accuracy: 0.9839 - val_loss: 0.0318 - val_accuracy: 1.0000
Epoch 16

23/23 [==============================] - 20s 890ms/step - loss: 0.1922 - accuracy: 0.9411 - val_loss: 0.1144 - val_accuracy: 0.9625
Epoch 3/20
23/23 [==============================] - 20s 890ms/step - loss: 0.1305 - accuracy: 0.9551 - val_loss: 0.0829 - val_accuracy: 0.9750
Epoch 4/20
23/23 [==============================] - 20s 887ms/step - loss: 0.1014 - accuracy: 0.9635 - val_loss: 0.0643 - val_accuracy: 0.9750
Epoch 5/20
23/23 [==============================] - 21s 899ms/step - loss: 0.0843 - accuracy: 0.9705 - val_loss: 0.0547 - val_accuracy: 0.9875
Epoch 6/20
23/23 [==============================] - 20s 889ms/step - loss: 0.0721 - accuracy: 0.9748 - val_loss: 0.0479 - val_accuracy: 0.9875
Epoch 7/20
23/23 [==============================] - 20s 886ms/step - loss: 0.0662 - accuracy: 0.9762 - val_loss: 0.0527 - val_accuracy: 0.9875
Epoch 8/20
23/23 [==============================] - 20s 886ms/step - loss: 0.0642 - accuracy: 0.9804 - val_loss: 0.0430 - val_accuracy: 0.9875
Epoch 9/20

22/22 [==============================] - 20s 888ms/step - loss: 0.0318 - accuracy: 0.9913 - val_loss: 0.0259 - val_accuracy: 1.0000
Epoch 17/20
22/22 [==============================] - 20s 888ms/step - loss: 0.0315 - accuracy: 0.9913 - val_loss: 0.0264 - val_accuracy: 1.0000
Epoch 18/20
22/22 [==============================] - 20s 887ms/step - loss: 0.0291 - accuracy: 0.9913 - val_loss: 0.0246 - val_accuracy: 1.0000
Epoch 19/20
22/22 [==============================] - 20s 893ms/step - loss: 0.0286 - accuracy: 0.9898 - val_loss: 0.0234 - val_accuracy: 1.0000
Epoch 20/20
22/22 [==============================] - 20s 888ms/step - loss: 0.0268 - accuracy: 0.9927 - val_loss: 0.0222 - val_accuracy: 1.0000
4/4 - 0s - loss: 0.0353 - accuracy: 0.9917

Test accuracy for airplane-curtain classes width res 24: 0.9916666746139526
0 12 ['airplane', 'desk']
Log file logs/fit/24-2-20200624-063546
Epoch 1/20
24/24 [==============================] - 21s 893ms/step - loss: 0.1884 - accuracy: 0.9139 - val_

...

17/17 [==============================] - 15s 888ms/step - loss: 0.0568 - accuracy: 0.9830 - val_loss: 0.0581 - val_accuracy: 0.9831
Epoch 9/20
17/17 [==============================] - 15s 877ms/step - loss: 0.0506 - accuracy: 0.9830 - val_loss: 0.0494 - val_accuracy: 0.9831
Epoch 10/20
17/17 [==============================] - 15s 877ms/step - loss: 0.0405 - accuracy: 0.9887 - val_loss: 0.0464 - val_accuracy: 1.0000
Epoch 11/20
17/17 [==============================] - 15s 876ms/step - loss: 0.0334 - accuracy: 0.9906 - val_loss: 0.0354 - val_accuracy: 0.9831
Epoch 12/20
17/17 [==============================] - 15s 874ms/step - loss: 0.0326 - accuracy: 0.9925 - val_loss: 0.0312 - val_accuracy: 1.0000
Epoch 13/20
17/17 [==============================] - 15s 874ms/step - loss: 0.0282 - accuracy: 0.9925 - val_loss: 0.0287 - val_accuracy: 1.0000
Epoch 14/20
17/17 [==============================] - 15s 881ms/step - loss: 0.0250 - accuracy: 0.9981 - val_loss: 0.0260 - val_accuracy: 1.0000
Epoch

Epoch 2/20
14/14 [==============================] - 12s 828ms/step - loss: 0.5379 - accuracy: 0.6859 - val_loss: 0.5860 - val_accuracy: 0.7872
Epoch 3/20
14/14 [==============================] - 12s 825ms/step - loss: 0.4597 - accuracy: 0.6930 - val_loss: 0.5554 - val_accuracy: 0.8298
Epoch 4/20
14/14 [==============================] - 12s 825ms/step - loss: 0.4723 - accuracy: 0.8082 - val_loss: 0.5363 - val_accuracy: 0.8085
Epoch 5/20
14/14 [==============================] - 12s 825ms/step - loss: 0.4052 - accuracy: 0.8249 - val_loss: 0.5284 - val_accuracy: 0.7872
Epoch 6/20
14/14 [==============================] - 12s 825ms/step - loss: 0.4413 - accuracy: 0.8465 - val_loss: 0.5185 - val_accuracy: 0.7872
Epoch 7/20
14/14 [==============================] - 12s 824ms/step - loss: 0.4284 - accuracy: 0.8225 - val_loss: 0.5239 - val_accuracy: 0.9574
Epoch 8/20
14/14 [==============================] - 12s 825ms/step - loss: 0.4063 - accuracy: 0.8297 - val_loss: 0.5116 - val_accuracy: 0.7447

Epoch 17/20
8/8 [==============================] - 7s 837ms/step - loss: 0.0710 - accuracy: 0.9725 - val_loss: 0.0610 - val_accuracy: 1.0000
Epoch 18/20
8/8 [==============================] - 7s 848ms/step - loss: 0.0688 - accuracy: 0.9686 - val_loss: 0.0592 - val_accuracy: 1.0000
Epoch 19/20
8/8 [==============================] - 7s 839ms/step - loss: 0.0677 - accuracy: 0.9765 - val_loss: 0.0550 - val_accuracy: 1.0000
Epoch 20/20
8/8 [==============================] - 7s 836ms/step - loss: 0.0626 - accuracy: 0.9765 - val_loss: 0.0523 - val_accuracy: 1.0000
4/4 - 0s - loss: 0.0975 - accuracy: 0.9583

Test accuracy for car-wardrobe classes width res 24: 0.9583333134651184
7 39 ['car', 'xbox']
Log file logs/fit/24-2-20200625-045710
Epoch 1/20
9/9 [==============================] - 7s 806ms/step - loss: 0.4549 - accuracy: 0.8037 - val_loss: 0.4139 - val_accuracy: 0.7667
Epoch 2/20
9/9 [==============================] - 7s 794ms/step - loss: 0.2161 - accuracy: 0.9222 - val_loss: 0.2703 - v

Epoch 11/20
28/28 [==============================] - 25s 893ms/step - loss: 0.0879 - accuracy: 0.9690 - val_loss: 0.0663 - val_accuracy: 0.9794
Epoch 12/20
28/28 [==============================] - 25s 893ms/step - loss: 0.0981 - accuracy: 0.9701 - val_loss: 0.0764 - val_accuracy: 0.9278
Epoch 13/20
28/28 [==============================] - 25s 892ms/step - loss: 0.0958 - accuracy: 0.9598 - val_loss: 0.0718 - val_accuracy: 0.9794
Epoch 14/20
28/28 [==============================] - 25s 895ms/step - loss: 0.0882 - accuracy: 0.9598 - val_loss: 0.0749 - val_accuracy: 0.9897
Epoch 15/20
28/28 [==============================] - 25s 891ms/step - loss: 0.0912 - accuracy: 0.9701 - val_loss: 0.0590 - val_accuracy: 0.9691
Epoch 16/20
28/28 [==============================] - 25s 893ms/step - loss: 0.0877 - accuracy: 0.9701 - val_loss: 0.0601 - val_accuracy: 0.9794
Epoch 17/20
28/28 [==============================] - 25s 892ms/step - loss: 0.0779 - accuracy: 0.9679 - val_loss: 0.0584 - val_accuracy:

29/29 [==============================] - 26s 910ms/step - loss: 0.1022 - accuracy: 0.9688 - val_loss: 0.0969 - val_accuracy: 0.9300
Epoch 5/20
29/29 [==============================] - 27s 934ms/step - loss: 0.0980 - accuracy: 0.9666 - val_loss: 0.0887 - val_accuracy: 0.9600
Epoch 6/20
29/29 [==============================] - 27s 944ms/step - loss: 0.0966 - accuracy: 0.9744 - val_loss: 0.0838 - val_accuracy: 0.9600
Epoch 7/20
29/29 [==============================] - 26s 899ms/step - loss: 0.0945 - accuracy: 0.9788 - val_loss: 0.1031 - val_accuracy: 0.9300
Epoch 8/20
29/29 [==============================] - 26s 889ms/step - loss: 0.0976 - accuracy: 0.9722 - val_loss: 0.0943 - val_accuracy: 0.9300
Epoch 9/20
29/29 [==============================] - 26s 888ms/step - loss: 0.1053 - accuracy: 0.9677 - val_loss: 0.1071 - val_accuracy: 0.9200
Epoch 10/20
29/29 [==============================] - 26s 888ms/step - loss: 0.0886 - accuracy: 0.9800 - val_loss: 0.0819 - val_accuracy: 0.9700
Epoch 11/

In [12]:
Y_pred = model.predict(prepare_data(test, len(base_labels)), len(test))
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_labels, y_pred))

Confusion Matrix
[[100   0]
 [  2  48]]


In [71]:
test, test_labels = load_data(res, all_labels, 'test', 1.0, False)


In [58]:
def real_index(prediction, i, j):
    return j if prediction else i

def prediction_for(i, j, data):
    model_name = 'models/' + model_type + '-' + all_labels[i] + '-' + all_labels[j] + '.h5'        
    model = keras.models.load_model(model_name)
    prediction = model.predict(data)
    return [real_index(p, i, j) for p in list(map(np.argmax, prediction))]


In [85]:
print(test_labels[0])
test[0].shape
to_test = np.expand_dims(test[0], axis=0)
#model.predict(to_test)

0.0


In [84]:
prediction = [x[:] for x in [[-1] * 40] * 40]
for i in range(len(all_labels)):
    for j in range(i + 1, len(all_labels)):
        prediction[i][j] = prediction_for(i, j, to_test)
        print(prediction[i][j])


[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[1]
[10]
[11]
[1]
[13]
[14]
[15]
[16]
[17]
[18]
[19]
[20]


[21]
[22]
[23]
[24]
[25]
[26]
[27]
[28]
[29]
[30]
[31]
[32]
[33]
[34]
[35]
[36]
[37]
[38]
[39]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[17]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[3]
[3]


[6]
[7]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[17]
[18]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[5]
[6]
[7]
[8]
[9]
[10]
[11]
[4]
[4]
[4]
[15]
[4]
[17]
[18]
[4]
[20]
[4]
[22]
[4]
[24]
[25]
[26]
[27]
[4]
[29]


[30]
[31]
[4]
[4]
[34]
[35]
[36]
[37]
[4]
[39]
[5]
[7]
[8]
[5]
[5]
[5]
[5]
[5]
[5]
[15]
[5]
[17]
[18]
[5]
[5]
[21]
[5]
[5]
[5]
[25]
[26]
[27]
[5]
[5]
[5]
[31]
[5]
[5]
[34]
[5]
[36]
[37]
[5]
[39]
[7]
[8]
[6]
[6]
[6]
[6]
[13]
[6]
[6]
[16]
[17]
[6]
[6]


[20]
[6]
[6]
[6]
[6]
[25]
[26]
[6]
[6]
[29]
[30]
[31]
[6]
[33]
[34]
[6]
[36]
[37]
[38]
[6]
[7]
[7]
[7]
[7]
[7]
[7]
[7]
[7]
[7]
[7]
[7]
[7]
[7]
[7]
[7]
[7]
[7]
[7]
[7]
[7]
[7]
[7]
[7]
[7]
[7]
[7]
[7]
[7]
[36]
[7]
[7]
[7]
[8]
[8]
[8]
[8]
[8]
[8]
[8]


...

[16]
[36]
[37]
[16]
[39]
[17]
[17]
[17]
[17]
[17]
[17]
[17]
[17]
[17]
[17]
[17]
[17]
[17]
[17]
[17]
[17]
[17]
[17]
[17]
[17]
[17]
[17]
[18]
[18]
[18]
[18]
[18]
[18]
[25]
[26]
[18]
[18]
[18]
[18]
[18]
[18]
[18]
[18]
[18]
[18]
[18]
[18]
[18]
[20]
[21]
[22]
[19]
[24]
[25]
[26]
[27]
[19]
[29]


[30]
[19]
[19]
[19]
[34]
[19]
[36]
[37]
[38]
[39]
[20]
[20]
[20]
[20]
[25]
[26]
[27]
[20]
[29]
[20]
[20]
[20]
[20]
[34]
[35]
[36]
[37]
[20]
[39]
[21]
[21]
[21]
[25]
[26]
[27]
[21]
[29]
[30]
[31]
[32]
[21]
[34]
[35]
[36]
[37]
[21]
[39]
[22]
[22]
[25]
[26]
[27]
[22]
[29]
[30]
[31]
[22]
[22]


[34]
[22]
[36]
[37]
[22]
[22]
[24]
[25]
[26]
[27]
[23]
[29]
[30]
[31]
[32]
[33]
[34]
[35]
[36]
[37]
[38]
[39]
[25]
[26]
[27]
[28]
[29]
[30]
[31]
[24]
[24]
[24]
[35]
[36]
[37]
[24]
[24]
[25]
[25]
[25]
[25]
[25]
[25]
[25]
[25]
[25]
[25]
[25]
[25]
[25]
[25]
[26]
[26]
[26]
[26]
[26]
[26]
[26]


[26]
[26]
[26]
[26]
[26]
[26]
[28]
[29]
[27]
[31]
[32]
[27]
[27]
[27]
[36]
[37]
[38]
[27]
[29]
[30]
[31]
[28]
[28]
[34]
[35]
[36]
[37]
[38]
[28]
[29]
[29]
[29]
[29]
[29]
[29]
[36]
[37]
[29]
[29]
[31]
[30]
[30]
[34]
[30]
[36]
[37]
[30]
[39]
[31]
[31]
[34]
[31]
[36]
[31]
[31]
[31]
[33]
[34]


[32]
[36]
[37]
[38]
[39]
[34]
[35]
[36]
[37]
[33]
[39]
[34]
[36]
[37]
[34]
[34]
[36]
[37]
[35]
[35]
[36]
[36]
[36]
[37]
[37]
[38]


In [119]:


arr = [item for sublist in prediction for item in sublist]
b = [a for a in arr]
np.delete(b, np.argwhere(b != -1))


array([-1, list([0]), list([0]), list([0]), list([0]), list([0]),
       list([0]), list([0]), list([0]), list([0]), list([0]), list([0]),
       list([0]), list([0]), list([0]), list([0]), list([0]), list([0]),
       list([0]), list([0]), list([0]), list([0]), list([0]), list([0]),
       list([0]), list([0]), list([0]), list([0]), list([0]), list([0]),
       list([0]), list([0]), list([0]), list([0]), list([0]), list([0]),
       list([0]), list([0]), list([0]), list([0]), -1, -1, list([2]),
       list([3]), list([4]), list([5]), list([6]), list([7]), list([8]),
       list([1]), list([10]), list([11]), list([1]), list([13]),
       list([14]), list([15]), list([16]), list([17]), list([18]),
       list([19]), list([20]), list([21]), list([22]), list([23]),
       list([24]), list([25]), list([26]), list([27]), list([28]),
       list([29]), list([30]), list([31]), list([32]), list([33]),
       list([34]), list([35]), list([36]), list([37]), list([38]),
       list([39]), -1, -1,

In [ ]:
num_classes = 20
res = 24
base_labels = all_labels[:num_classes]
test, test_labels = load_data(res, base_labels, 'test', 1.0, False)


In [ ]:
test_loss, test_acc = model.evaluate(
            prepare_data(test, num_classes),  
            test_labels, 
            verbose=2
        )
print('\nTest accuracy for ' + str(num_classes) + ' classes width res ' + str(res) + ':', test_acc)
print_confusion_matrix(test, test_labels, base_labels)

In [ ]:
def draw(img, expected = '', predicted = ''):
    fig = plt.figure(figsize=[10,10])
    ax = fig.gca(projection='3d')
    cube1 = (img[:,:,:] >= 1)
    ax.voxels(cube1, facecolors="blue")
    plt.title('Expected: {}\n Predicted: {}'.format(expected, predicted), y=-0.01)
    plt.show()

In [ ]:
if not len(test):
    test, test_labels = load_data(dim, base_labels, 'test', 100)
test_loss, test_acc = model.evaluate(test,  test_labels, verbose=2)
p = model.predict(test)
print('\nTest accuracy:', test_acc)

In [ ]:
index = random.randint(0, len(test))

predicted_index = np.argmax(p[index])
expected = base_labels[int(test_labels[index])]
predicted = base_labels[predicted_index]
draw(test[index], expected, predicted)
index

In [ ]:
num_classes = 3
base_labels = all_labels[:num_classes]
num_classes = num_classes
res = 24
training, training_labels = load_data(res, base_labels, 'train', 1, True)


In [ ]:
print(len(training))
model = prepare_model(res, num_classes, 1)
train(model, training, training_labels, res, num_classes, 0)


In [ ]:
print(base_labels)
test, test_labels = load_data(res, base_labels, 'test', 1.0, True)
print(len(test))
test_loss, test_acc = model.evaluate(
    test,  
    test_labels, 
    verbose=2
)


In [ ]:
p = model.predict(test)
for index in range(len(p)):
    predicted_index = np.argmax(p[index])
    expected = base_labels[int(test_labels[index])]
    predicted = base_labels[predicted_index]
    if expected != predicted:
        draw(test[index], expected, predicted)
